# Imports

In [140]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
import torch.nn as nn
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon, LinearRing

# Input

In [141]:
torch.manual_seed(0)  #  for repeatable results
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [142]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [143]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [144]:
class DevConv(nn.Module):
    def __init__(self, nodes, adjacency_matrix, output_dimension):
        super().__init__()
        self.size = output_dimension
        self.nodes = nodes
        self.adjacency_matrix = adjacency_matrix
        self.W_phi = nn.Parameter(torch.Tensor(output_dimension))
        self.W_theta = nn.Parameter(torch.Tensor(size=(3,1)))

        nn.init.normal_(self.W_phi)
        nn.init.normal_(self.W_theta)

        # print("self.W_phi.shape : ", self.W_phi.shape)
        # print("self.W_theta.shape : ", self.W_theta.shape)
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = torch.zeros((self.nodes.shape[0], self.size))                                          #list of "output_dimension" for each "list_node" element
        for index_current_node, list_neighbors in enumerate(self.adjacency_matrix):                             # for each node and its adjacency nodes
            neighbors = self.nodes[list_neighbors.nonzero()]                                                    # get neighbors nodes
            diff = self.nodes[index_current_node] - neighbors                                                   # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            to_norm = self.W_theta.T.unsqueeze(1).repeat(1, diff.shape[0], 1)[0] * diff.squeeze(1)              # Compute W_theta * (x_i - x_j)
            neigh_distances = torch.norm(to_norm, dim=1)                                                        # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = (self.W_phi * neigh_distances.max()).clone()                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = torch.mean(list_inc_score, dim=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        result_np = torch.stack([previous_inclusion_score, torch.tensor(list_inc_score)])
        
        result_np = torch.mean(result_np, dim=0)
        
        return result_np


class GNN_Model(nn.Module):
    def __init__(self, nodes, adjacency_matrix):
        super(GNN_Model, self).__init__()
        self.devconv = DevConv(nodes, adjacency_matrix, 1)
        self.relu = nn.ReLU()
        self.devconv2 = DevConv(nodes, adjacency_matrix, 64)
        self.relu2 = nn.ReLU()
        self.devconv3 = DevConv(nodes, adjacency_matrix,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x= self.devconv(x)
        x= self.relu(x)
        x= self.devconv2(x)
        x= self.relu2(x)
        x= self.devconv3(x)
        x= self.sigmoid(x)
        return x


gnn = GNN_Model(torch.Tensor(np.array(graph)), torch.Tensor(nx.adjacency_matrix(graph).toarray()))
inclusion_score = gnn(torch.empty(0))
print(inclusion_score.shape)

C:\Users\ahdel\AppData\Local\Temp\ipykernel_4884\1471860285.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result_np = torch.stack([previous_inclusion_score, torch.tensor(list_inc_score)])


torch.Size([5999])


### Multinomial Sampling

In [145]:
class MultinomialLayer(nn.Module):
    def __init__(self, target_number_points,nodes):
        super().__init__()
        self.target_number_points = target_number_points
        self.nodes = nodes

    def forward(self, f):
        normalized_inclusion_score = f / torch.sum(f)                           # normalize for multinomial sampling

        mult_sampling = torch.distributions.multinomial.Multinomial(total_count=10*normalized_inclusion_score.shape[0], probs=normalized_inclusion_score).sample()      # small:more randomness    |   big:less randomness
        mult_indices = mult_sampling.topk(k=self.target_number_points).indices
        selected_nodes = self.nodes[mult_indices]

        return selected_nodes


target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification
layer = MultinomialLayer(target_number_point, torch.Tensor(np.array(graph)))
extended_graph_nodes = layer.forward(torch.Tensor(inclusion_score))
print(extended_graph_nodes.shape)

torch.Size([1500, 3])


# KNN extended graph

In [146]:
from torch.nn.functional import pairwise_distance

def create_knn_graph(nodes, k):
    """
    Create a graph based on k-nearest neighbors using PyTorch.
    Parameters:
    - nodes: Tensor of shape (n, 3) representing 3D nodes.
    - k: Number of nearest neighbors.
    Returns:
    - adjacency_matrix: Binary adjacency matrix representing the graph.
    """
    expanded_x1 = nodes.unsqueeze(1)
    expanded_x2 = nodes.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)        # distance matrix

    _, indices = torch.topk(distances, k + 1, largest=False, sorted=True, dim=1)
    indices = indices[:, 1:]  # Exclude the node itself

    # Create adjacency matrix
    adjacency_matrix = torch.zeros(nodes.shape[0], nodes.shape[0], dtype=torch.float32)
    adjacency_matrix.scatter_(1, indices, 1)

    return adjacency_matrix

extended_graph_adjacency_matrix = create_knn_graph(extended_graph_nodes, k=15)
transformation.print_graph_properties(graph=nx.from_numpy_array(extended_graph_adjacency_matrix.numpy()), display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 12612


In [147]:
extended_graph_nodes.shape

torch.Size([1500, 3])

# Edge Predictor

In [148]:
devconv = DevConv(extended_graph_nodes,extended_graph_adjacency_matrix, 64)
inclusion_score = devconv(previous_inclusion_score=torch.empty((0)), return_flatten=False)
inclusion_score.shape

torch.Size([1500, 64])

In [149]:
class SparseAttentionEdgePredictorLayer(nn.Module):
    def __init__(self, nodes, neighbors, size=64):
        super().__init__()
        self.size = size
        self.nodes = nodes
        self.neighbors = neighbors
        self.wq = nn.Parameter(torch.Tensor(size))
        self.wk = nn.Parameter(torch.Tensor(size))

        nn.init.normal_(self.wq)
        nn.init.normal_(self.wk)

    def forward(self, f):
        wq_f = self.wq.reshape(-1, 1) * f                   # Wq*f
        wk_f = self.wk.reshape(-1, 1) * f                   # Wq*f
        S = torch.exp(torch.matmul(wq_f.T, wk_f))           # e^((wq_f.T)*(wk_f))
        
        nonzero_neigh = self.neighbors.nonzero()                                                    # Find indexes of neighbors in graph
        unique_first_elements, counts = torch.unique(nonzero_neigh[:, 0], return_counts=True)       # Count number of neighbors per node
        split_tensors = list(torch.split(nonzero_neigh, tuple(counts)))                             # split indexes of neighbors into a list (1 element = 1 tensor of indexes)

        temp = [[S[n[i,0], n[i,1]] for i in range(len(n))] for n in split_tensors]                  # For each node, get the S value for the neighbors indexes
        summed = torch.Tensor([torch.sum(torch.Tensor(e)) for e in temp])                           # Sum these results for each nodes
        division = summed.unsqueeze(0).repeat(1, S.shape[1], 1)[0]                                  # Repeat the sum in S.shape[1] array => division per columns
        final_term  = S / division

        return final_term


f = torch.mean(inclusion_score, dim=1)                            # Flatten the matrix of inclusion score
layer = SparseAttentionEdgePredictorLayer(extended_graph_nodes, extended_graph_adjacency_matrix)
S = layer.forward(torch.Tensor(f))
print(S.shape)

torch.Size([1500, 1500])


### Sparse Attention

In [150]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [151]:
class FaceCandidatesLayer(nn.Module):
    def __init__(self, adjacency_matrix):
        super().__init__()
        self.adjacency_matrix = adjacency_matrix

    def forward(self, S):
        A_s = torch.matmul(torch.matmul(S, self.adjacency_matrix), S.T)     # A_s = S * A * S.T
        A_s = A_s/A_s.max()                                                 # Normalize
        return A_s


layer = FaceCandidatesLayer(extended_graph_adjacency_matrix)
A_s = layer(torch.Tensor(S))
print(A_s.shape)

torch.Size([1500, 1500])


# Face Classifier

### TriConv

#### Inputs

In [152]:
igraph_g = igraphGraph(directed=False).from_networkx(nx.from_numpy_array(extended_graph_adjacency_matrix.numpy()))
print(igraph_g.summary())

IGRAPH U-W- 1500 12612 -- 
+ attr: _nx_name (v), weight (e)


In [153]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
triangles = extended_graph_nodes[triangles_ids_igraph]
print(triangles.shape)

torch.Size([43747, 3, 3])


In [182]:
nonzero = extended_graph_adjacency_matrix.nonzero()
# print(nonzero.shape)
neighbors_one_indexes = nonzero.reshape(1500,15,2)[:,:,1].clone()
# print("neighbors_indexes : ", neighbors_one_indexes.shape, neighbors_one_indexes)

neighbors_two_indexes = neighbors_one_indexes[neighbors_one_indexes]
# neighbors_two_indexes = neighbors_two_indexes[:,:,1:]                                                   # remove returns from the one neighbors
# print("neighbors_two_indexes : ", neighbors_two_indexes.shape, neighbors_two_indexes)

neighbors_three_indexes = neighbors_one_indexes[neighbors_two_indexes]
# print("neighbors_three_indexes : ", neighbors_three_indexes.shape, neighbors_three_indexes)

# Find the indices where the current index is present along the last dimension
values_index_reshape = torch.arange(neighbors_three_indexes.shape[0]).repeat((15,15,15,1)).T
# print("temp : ", values_index_reshape.shape)
indices = (neighbors_three_indexes == values_index_reshape).nonzero()
# print(indices.shape)
# print(indices)

In [183]:
i, j, k, l = indices[:,0], indices[:,1], indices[:,2], indices[:,3]
temp_j = neighbors_one_indexes[i,j]
temp_k = neighbors_two_indexes[i,j,k]
temp_l = neighbors_three_indexes[i,j,k,l]
triangles_indexes_test = torch.stack((i, temp_j, temp_k, temp_l), dim=1)
triangles_indexes_test = triangles_indexes_test[:,:3]               # remove virtual 4th point
print(triangles_indexes_test)

In [ ]:
# filter triangles indexes to clean the clones (=> divide the number of triangles by 6)

In [181]:
extended_graph_nodes[triangles_indexes_test].shape

torch.Size([196926, 3, 3])

In [15]:
class FirstPInitLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, triangles_indexes):
        # Extract indices for each triangle
        i, j, k = triangles_ids_igraph.T

        # Extract probabilities using advanced indexing
        A_s_ij = A_s[i, j]
        A_s_ik = A_s[i, k]
        A_s_jk = A_s[j, k]

        # Calculate the barycenter probabilities
        p_init = torch.zeros(len(triangles))
        p_init = (A_s_ij + A_s_ik + A_s_jk) / 3
        return p_init

p_init_layer = FirstPInitLayer()
p_init = p_init_layer(triangles_ids_igraph)
print(p_init.shape)

torch.Size([43747])


#### Calculate barycenter

In [16]:
class BarycentersLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, triangles):
        return triangles.mean(1)

barycenters_layer = BarycentersLayer()
barycenters = barycenters_layer(triangles)
print(barycenters.shape)

torch.Size([43747, 3])


#### KNN Tri

In [17]:
class KNN(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, k=20):
        indices_knn = torch.empty(size=(x.shape[0], k))
        
        for i, neighbors in enumerate(x):
            distances = torch.norm(neighbors-x, dim=1)
            # print(distances.shape)
            # print(distances)
            # print(i)

            indices_knn[i] = distances.topk(k, dim=0, largest=False).indices.clone()  # Indices of the k-nearest neighbors
            # print(indices_knn.shape)
            # print(indices_knn)
            
        return indices_knn

knn_layer = KNN()
indices_neigh_tri = knn_layer(barycenters).int()  #change datatype
print(indices_neigh_tri.shape)

torch.Size([43747, 20])


In [18]:
class RMatrix(nn.Module):
    def __init__(self, triangles, barycenters, indices_neigh_tri):
        super().__init__()
        self.triangles = triangles
        self.barycenters = barycenters
        self.indices_neigh_tri = indices_neigh_tri

    def forward(self):
        # DIFF BARYCENTERS
        barycenters_diff = np.subtract(self.barycenters[self.indices_neigh_tri[:, 0]][:, np.newaxis], self.barycenters[self.indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire


        # TRIANGLE EDGES NORM
        v0, v1, v2 = self.triangles[:, 0], self.triangles[:, 1], self.triangles[:, 2]

        # Calculate edge vectors
        e_ij = torch.norm(v0 - v1, dim=1)
        e_ik = torch.norm(v0 - v2, dim=1)
        e_jk = torch.norm(v1 - v2, dim=1)

        # Stack the edge vectors along the last dimension
        diff_vectors = torch.stack([e_ij, e_ik, e_jk], dim=1)


        # MAX/MIN DIFF VECTORS
        max_diff_vectors = diff_vectors.max(dim=1).values       # calculate t_n_max
        min_diff_vectors = diff_vectors.min(dim=1).values       # calculate t_n_min

        max_diff_vectors_diff = max_diff_vectors[self.indices_neigh_tri[:, 0]][:, None] - max_diff_vectors[self.indices_neigh_tri[:, 1:]]   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
        min_diff_vectors_diff = min_diff_vectors[self.indices_neigh_tri[:, 0]][:, None] - min_diff_vectors[self.indices_neigh_tri[:, 1:]]   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min


        # R MATRIX COMPUTATION
        r_matrix = torch.zeros((self.triangles.shape[0], number_neigh_tri-1, 5))

        r_matrix[:, :, 0]   = min_diff_vectors_diff
        r_matrix[:, :, 1]   = max_diff_vectors_diff
        r_matrix[:, :, 2:5] = barycenters_diff
        
        return r_matrix


r_matrix_layer = RMatrix(triangles, barycenters, indices_neigh_tri)
r_matrix = r_matrix_layer()
r_matrix.shape

torch.Size([43747, 19, 5])

#### Calculate f

In [ ]:
class MLP(nn.Module):
  def __init__(self, r_matrix):
    super().__init__()
    self.r_matrix = r_matrix
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(32 * 32 * 3, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 10)
    )


  def forward(self, x):
    
    return self.layers(x)

f_init = p_init
mlp = MLP(r_matrix)


In [50]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.99999964
(43747,)
[2.3804538e-05 2.3797165e-05 2.3302142e-05 ... 2.1143813e-05 2.1072487e-05
 2.1078808e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_6276\474768578.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
C:\Users\Arthur\AppData\Local\Temp\ipykernel_6276\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [ ]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[ 51.         -3.0965528 -13.237194 ]
 [ 39.         -8.699333   -3.6727462]
 [ 51.          4.590885    2.507581 ]]


In [ ]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 43
Number of edges: 203


In [ ]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

In [ ]:
def torch_d_P_Ps(p_y, x, y):
    """All Tensors in input"""
    # print(p_y.shape, x.shape, y.shape)

    expanded_x1 = x.unsqueeze(1)
    expanded_x2 = y.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)        # distance matrix

    min_x = distances.min(dim=1).values
    min_y = distances.min(dim=0)

    first_term = torch.sum(torch.index_select(p_y, 0, min_y.indices) * min_y.values)
    second_term = torch.sum(min_x * p_y)

    return first_term + second_term


d_P_Ps = torch_d_P_Ps(torch.Tensor(normalized_inclusion_score), torch.Tensor(np.array(graph)), torch.Tensor(np.array(extended_graph)))

## Probabilistic Surfaces Distance

### d_f_S_Ss

In [ ]:
def torch_d_f_S_Ss(p_b_hat, b_hat, b):
    # print(p_b_hat.shape, b_hat.shape, b.shape)

    expanded_x1 = b_hat.unsqueeze(1)
    expanded_x2 = b.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)

    min_b = distances.min(dim=1).values

    final_term = torch.sum(p_b_hat * min_b)

    return final_term



igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
torch_b = torch.Tensor(np.mean(triangles, axis=1))

torch_b_hat = torch.Tensor(np.mean(selected_triangles, axis=1))

torch_p_b_hat = torch.Tensor(final_scores[selected_triangles_indexes])

d_f_S_Ss = torch_d_f_S_Ss(torch_p_b_hat, torch_b_hat, torch_b)

### d_r_S_Ss

In [ ]:
def torch_d_r_S_Ss(p_x, p_y, x ,y):
    expanded_x = x.unsqueeze(1)
    expanded_y = y.unsqueeze(0)
    distances = torch.norm(expanded_x - expanded_y, dim=2)
    min_d = distances.min(dim=0).values
    first_term = p_y * min_d


    indices_knn = distances.topk(k=50, dim=0, largest=False).indices.T  # Indices of the k-nearest neighbors
    knn_labels = x[indices_knn]
    xtk = torch.reshape(knn_labels, shape=((y.shape[0])*50, 3))

    expanded_xtk = xtk.unsqueeze(1)
    distances_knn = torch.norm(expanded_xtk - expanded_y, dim=2)
    min_knn = distances_knn.min(dim=1).values
    min_knn_reshaped = min_knn.reshape(((y.shape[0]), 50))

    ptk_time_norm = p_x[indices_knn] * min_knn_reshaped
    factor = (1-p_y) * (1/50)
    second_term = factor * torch.sum(ptk_time_norm, dim=1)

    final_term = torch.sum(first_term + second_term)

    return final_term

d_f_S_Ss = torch_d_r_S_Ss(torch.Tensor(final_scores), torch_p_b_hat, torch_b, torch_b_hat)
d_f_S_Ss

tensor(0.0768)

## Triangle Collision Loss

In [ ]:
def compute_lc_le_lo(p_t, m_c_e_o, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_e_o), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_e_o

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c_e_o = (1 / len(Fs)) * total_penalty

    return L_c_e_o

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
Fs = triangles  # Given data

In [ ]:
number_neigh_barycenters = min(50, len(b_hat))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_barycenters).fit(b_hat).kneighbors(b_hat)

NameError: name 'b_hat' is not defined

### Lc

In [ ]:
mc = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    polygons_others_tri = MultiPolygon([Polygon(others_triangle) for others_triangle in others_triangles]).buffer(0)    # buffer 0 to correct invalid polygons => take the exterior of the shape


    intersection = lines_current_triangle.intersection(polygons_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        mc[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        mc[index_neigh_barycenters[0]] += 1
L_c = compute_lc_le_lo(p_t, mc, Fs)
print("L_c : ", L_c)

L_c :  4225.18776054694


### Le

In [ ]:
me = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    lines_others_tri = MultiLineString([LineString([other_tri[0], other_tri[1], other_tri[2], other_tri[0]]) for other_tri in others_triangles])

    intersection = lines_current_triangle.intersection(lines_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        me[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        me[index_neigh_barycenters[0]] += 1
L_e = compute_lc_le_lo(p_t, me, Fs)
print("L_e : ", L_e)

L_e :  24105.658829414708


### Lo

In [ ]:
mo = np.zeros((500))

Lo - échantillonnage de 100 points à partir de chaque triangle

In [ ]:

def sample_points_from_triangle(t, num_points=100):
    v1, v2, v3 = t
    bary_coords = np.random.rand(num_points, 2)
    sqrt_bary_coords = np.sqrt(bary_coords[:, 0])

    u = sqrt_bary_coords
    v = bary_coords[:, 1]

    """
    La formule spécifique est dérivée de l'expression générale d'interpolation barycentrique 
    sommets A, B et C
    coord barycentriques: u et v
    coord cartésiennes: x,y,z 
    """
    x_coords = (1 - u - v) * v1[0] + u * v2[0] + v * v3[0]
    y_coords = (1 - u - v) * v1[1] + u * v2[1] + v * v3[1]
    z_coords = (1 - u - v) * v1[2] + u * v2[2] + v * v3[2]

    sampled_points = np.column_stack((x_coords, y_coords, z_coords))
    return sampled_points

points100 = sample_points_from_triangle(triangle, num_points=100)
points100
points100.shape

(100, 3)

Les Aires

In [ ]:
number_neigh_selected_barycenters = min(50, len(b_hat))
def knnbar(nn):
  _, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=nn).fit(b_hat).kneighbors(b_hat)
  return _, indexes_neigh_selected_barycenters

In [ ]:
d, indices = knnbar(number_neigh_selected_barycenters)
indices

array([[  0, 292,  20, ..., 181, 180, 141],
       [  1,   6, 344, ..., 397, 151, 473],
       [  2, 186,  10, ..., 185, 184, 189],
       ...,
       [497, 421, 418, ..., 379, 238, 457],
       [498, 261, 303, ..., 385,  40, 327],
       [499, 264, 126, ..., 404, 338, 320]], dtype=int64)

In [ ]:
from scipy.spatial import distance as scipy_distance

def calculate_triangle_area(triangle):
    # Fonction pour calculer l'aire d'un triangle en utilisant la formule de Héron
    side_lengths = [scipy_distance.euclidean(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    s = sum(side_lengths) / 2
    return np.sqrt(s * np.prod([s - length for length in side_lengths]))

def penalize_overlapping_triangles(points_list, triangles, k=50):
    # Fonction pour pénaliser les triangles qui se chevauchent
    assignment_results = []
    overlapping_triangles = np.zeros(len(triangles), dtype=int)  # Déclaration en dehors de la boucle

    # ajustement de la valeur maximale de k en fonction du nombre de triangles
    k = min(k, len(triangles))

    # NearestNeighbors pour trouver les k triangles les plus proches pour chaque point
    knn_model = NearestNeighbors(n_neighbors=k).fit(np.vstack(triangles))
    d, indices = knn_model.kneighbors(np.array(points_list)[:, 0, :])

    for i, point_list in enumerate(points_list):
        point = point_list[0]

        # Verif que les indices sont valides
        closest_triangle_indices = indices[i, :k]
        closest_triangle_indices = closest_triangle_indices[closest_triangle_indices < len(triangles)]

        # Accéder aux triangles en utilisant les indices valides
        valid_triangle_indices = []
        for idx in closest_triangle_indices:
            modified_triangle = np.copy(triangles[idx])
            modified_triangle[0] = point

            area_original = calculate_triangle_area(triangles[idx])
            area_modified = calculate_triangle_area(modified_triangle)

            # Vérifier si la somme des aires est proche de l'aire du triangle
            if not np.isclose(area_original, area_modified, rtol=1e-5):
                assignment_results.append((point, triangles[idx]))
                valid_triangle_indices.append(idx)

        # MAJ du overlapping_triangles avec les indices valides
        overlapping_triangles[valid_triangle_indices] += 1

    penalties = overlapping_triangles
    total_penalty = np.sum(penalties)
    Lo = (1 / len(triangles)) * total_penalty

    return Lo


L_o = penalize_overlapping_triangles([points100], triangles, k=50)
print("Pénalité pour les triangles qui se chevauchent :", L_o)


Pénalité pour les triangles qui se chevauchent : 0.0008337502084375521


## Final Loss

lambda_c, lambda_e, lambda_o = 0.01, 0.01, 0.01


Loss_L = prob_chamfer_dist + d_f_S_Ss + d_r_S_Ss + (lambda_c * L_c) + (lambda_e * L_e) + (lambda_o * L_o)
print(Loss_L)